# Generating NN Code Bench OOB Extension

This is an experimental process to expand the smaller sized samples of NN Code Bench for the purpose of creating a dataset that can be used to train NN models. The samples that are going to be used to expand the dataset, through fuzzing techniques, are the **Polynomial Approximation Networks**, the **Hopfield Nets**, the **Reach Prob Density**, and the **Reinforcement Learning** samples. The **Activation Functions** and the **Math Functions** will not be used as they are too small or don't have any code that can cause memory leaks if manipulated (Dont use math and activation categories as they dont contain array out of bounds (They are too small)).

The code extensions will focus on array out of bounds, as it is an easy safety property to detect.

* Timeout: 900s as in SVComp - 30s for testing.

The following research questions are going to be answered by the expansion of the dataset:

* What size can ESBMC handle?
* Will ESBMC be able to find the bug in a reasonable time? Compare 30s vs 900s.
* How do other software versifiers perform with the task of solving the expanded dataset?

## TODO

- Remove assert safety properties by Eduardo in the code.


## Preprocess Data

This notebook takes care of processing the data of NN Codebench by compiling them into executables. Takes the programs in `networks` and compiles them to `bin` in the same structure.

### Base Dataset

#### Building Base Source Dataset

Building it from scratch.

In [13]:
import os
import shutil
from contextlib import chdir

In [14]:
# Create the build folder in case it does not exist.
BUILD_PATH: str = "./build"
if os.path.exists(BUILD_PATH):
    print(f"{BUILD_PATH} already exists.")
else:
    os.makedirs(BUILD_PATH)
    print(f"Created {BUILD_PATH}")

./build already exists.


In [15]:
# Build all base benchmarks.
with chdir(BUILD_PATH):
    print("Building all samples...")
    print(os.popen("cmake ..").read())
    print(os.popen("make -j4 install").read())
    print("Completed...")

Building all samples...
-- The C compiler identification is GNU 12.2.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /home/yiannis/git/plain_c_nn_benchmark/build

[  0%] Building C object includes/CMakeFiles/keras_lib.dir/keras2c/k2c_activations.c.o
[  0%] Building C object networks/CMakeFiles/reinforcement_lib.dir/reinforcement_learning/cartpole.c.o
[  0%] Building C object includes/CMakeFiles/stub.dir/verifier_stubs.c.o
[  1%] Building C object networks/CMakeFiles/reach_prob_lib.dir/reach_prob_density/gcas.c.o
[  1%] Linking C static library libstub.a
[  1%] Building C object includes/CMakeFiles/keras_lib.dir/keras2c/k2c_convolution_layers.c.o
[  1%] Building C object networks/CMakeFiles/reinforcement_lib.dir/reinforcement_learning/dubinsrejoin.c.o
[  1%] Bui

#### Remove intermediate files (speed up processing).

In [16]:
if os.path.exists(BUILD_PATH):
    count: int = 0
    with chdir(BUILD_PATH):
        files: list[str] = os.listdir(".")
        for file in files:
            if os.path.isfile(file) and file.endswith((".c", ".i", ".yml")):
                os.remove(file)
                count += 1
    print(f"Cleaned up {count} intermediate files.")

Cleaned up 1827 intermediate files.


In [17]:
build_folders: list[str] = [
    f"{BUILD_PATH}/export/hopfield_nets",
    f"{BUILD_PATH}/export/poly_approx",
    f"{BUILD_PATH}/export/reach_prob_density",
    f"{BUILD_PATH}/export/reinforcement_learning",
]

#### Extracting it from SV-Bench

They mirror the built ones. So if we want to use the ones from SV-Bench, we can do so by
looking at the manually built files names (for example poly-approx) and transfering the
ones we want from the SV-Bench folder to the built one (replacing the manually built ones).

Why? Because of [this](https://github.com/emanino/plain_c_nn_benchmark/issues/24).

Create a `sv-bench` folder in the root of the project with the contents of `sv-bench/c/neural-networks`.

In [19]:
def transfer_from_sv_bench(dest_path: str, delete_source=False) -> None:
    """Transfers the files from SV Bench to the dest path. If `delete_source` is True
    then the transfer will also remove from the SV_Bench folder."""
    SV_BENCH_FOLDER = "sv-bench"
    print(f"Moving files from {SV_BENCH_FOLDER} to {dest_path}")
    # The folder for SV-Bench neural network samples.
    if not os.path.exists(SV_BENCH_FOLDER):
        raise FileNotFoundError(f"Directory SV-Bench does not exist: {SV_BENCH_FOLDER}")
    if not os.path.exists(dest_path):
        raise FileNotFoundError(f"Directory {dest_path} does not exist.")

    count: int = 0
    for idx, file in enumerate(os.listdir(dest_path)):
        src: str = f"{SV_BENCH_FOLDER}/{file}"
        dst: str = f"{dest_path}/{file}"
        if os.path.isfile(src) and file.endswith(".c"):
            shutil.copy(src=src, dst=dst)
            print(f"{idx}: {file}")
            count += 1

    print(f"Completed moving {count} files from {SV_BENCH_FOLDER} to {dest_path}\n")

# Copy SV-Bench files to each folder used.
for folder in build_folders:
    transfer_from_sv_bench(folder)

Moving files from sv-bench to ./build/export/hopfield_nets
2: tanh_w4_r1_case_0_unsafe.c-amalgamation.c
5: tanh_w4_r1_case_1_safe.c-amalgamation.c
8: tanh_w4_r2_case_0_safe.c-amalgamation.c
11: tanh_w4_r2_case_1_unsafe.c-amalgamation.c
14: tanh_w4_r3_case_0_safe.c-amalgamation.c
17: tanh_w4_r3_case_1_unsafe.c-amalgamation.c
20: tanh_w4_r4_case_0_safe.c-amalgamation.c
23: tanh_w4_r4_case_1_unsafe.c-amalgamation.c
26: tanh_w8_r1_case_0_unsafe.c-amalgamation.c
29: tanh_w8_r1_case_1_safe.c-amalgamation.c
32: tanh_w8_r2_case_0_safe.c-amalgamation.c
35: tanh_w8_r2_case_1_unsafe.c-amalgamation.c
38: tanh_w8_r3_case_0_safe.c-amalgamation.c
41: tanh_w8_r3_case_1_unsafe.c-amalgamation.c
44: tanh_w8_r4_case_0_safe.c-amalgamation.c
50: tanh_w16_r1_case_0_unsafe.c-amalgamation.c
53: tanh_w16_r1_case_1_safe.c-amalgamation.c
56: tanh_w16_r2_case_0_safe.c-amalgamation.c
59: tanh_w16_r2_case_1_unsafe.c-amalgamation.c
62: tanh_w16_r3_case_0_safe.c-amalgamation.c
65: tanh_w16_r3_case_1_safe.c-amalgamatio

### Prepare and Build Base Samples

We now have the base samples built. So we need to mutate and build base samples.

In [20]:
def build_sample(sample_file_path: str) -> None:
    # Add Mull compiler plugin.
    MULL: str = "-fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line"

    if not os.path.exists(sample_file_path):
        raise FileExistsError(f"No file found in {sample_file_path}")
    
    sample_name: str = os.path.basename(sample_file_path)

    if not sample_name.endswith(".c"):
        raise ValueError(f"{sample_name} is not a C source code filename.")

    if not os.path.exists("bin"):
        raise FileNotFoundError("No bin folder...")
    
    ver_h: str = "includes"
    poly_aprox_h: str = "networks"

    source_files: list[str] = ["includes/verifier_stubs.c", sample_file_path]
    object_files: list[str] = ["bin/intermediate/" + os.path.basename(source_file)[:-2] + ".o" for source_file in source_files]
    
    # Build each file into an object file (-c) and then combine. This is due to the Mull plugin,
    # which only works on single files.
    for source_file, object_file in zip(source_files, object_files):
        cmd: str = f"clang-15 {MULL} -c {source_file} -I{ver_h} -I{poly_aprox_h} -o {object_file}"
        print("\n" + cmd)
        print(os.popen(cmd).read())
    else:
        print()

    out: str = f"bin/poly_approx/{sample_name.removesuffix('.c')}"

    # Link
    cmd: str = f"clang-15 {MULL} {' '.join(object_files)} -I{ver_h} -I{poly_aprox_h} -lm -o {out}"
    print(cmd)
    print(os.popen(cmd).read())

    print(f"Compilation Complete: {out}")

In [21]:
count: int = 0
for folder in build_folders:
    print("Compiling folder:", folder)
    for file in os.listdir(folder):
        file_path: str = f"{folder}/{file}"
        if os.path.isfile(file_path) and file.endswith(".c"):
            print(f"Building {count} {file_path}")
            build_sample(file_path)
            print()
            count += 1
    print()

print(f"Compilation of samples has completed: Total {count}...")
del count

Compiling folder: ./build/export/hopfield_nets
Building 0 ./build/export/hopfield_nets/tanh_w4_r1_case_0_unsafe.c-amalgamation.c

clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line -c includes/verifier_stubs.c -Iincludes -Inetworks -o bin/intermediate/verifier_stubs.o
[info] Using configuration /home/yiannis/git/plain_c_nn_benchmark/mull.yml
[warning] Invalid regex for exclude-path: '^(?=.*\/(networks)\/).*$' has been ignored. Error: repetition-operator operand invalid
[info] Found compilation flags in the input bitcode
[info] Gathering functions under test (threads: 1)

       [################################] 1/1. Finished in 0ms
[info] Instruction selection (threads: 3)

       [##########----------------------] 1/3
       [################################] 3/3. Finished in 10ms
[info] Searching mutants across functions (threads: 3)

       [################################] 3/3. Finished in 10ms
[info] Applying filter: no d

./build/export/hopfield_nets/tanh_w8_r4_case_1_safe.c-amalgamation.c:9:62: error: expected ';' at end of declaration
    float input_array[32] = {0.0f}, output_array[32] = {0.0f}
                                                             ^
                                                             ;
./build/export/hopfield_nets/tanh_w8_r4_case_1_safe.c-amalgamation.c:10:62: error: expected ';' at end of declaration
        k2c_tensor input_tensor = {&input_array[0],2,32,{4,8,1,1,1}}
                                                                    ^
                                                                    ;
./build/export/hopfield_nets/tanh_w8_r4_case_1_safe.c-amalgamation.c:11:64: error: expected ';' at end of declaration
        k2c_tensor output_tensor = {&output_array[0],2,32,{4,8,1,1,1}}
                                                                      ^
                                                                      ;
./build/export/hopfield_nets/tanh_w



clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line bin/intermediate/verifier_stubs.o bin/intermediate/tanh_w8_r4_case_1_safe.c-amalgamation.o -Iincludes -Inetworks -lm -o bin/poly_approx/tanh_w8_r4_case_1_safe.c-amalgamation

Compilation Complete: bin/poly_approx/tanh_w8_r4_case_1_safe.c-amalgamation

Building 16 ./build/export/hopfield_nets/tanh_w16_r1_case_0_unsafe.c-amalgamation.c

clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line -c includes/verifier_stubs.c -Iincludes -Inetworks -o bin/intermediate/verifier_stubs.o
[info] Using configuration /home/yiannis/git/plain_c_nn_benchmark/mull.yml
[warning] Invalid regex for exclude-path: '^(?=.*\/(networks)\/).*$' has been ignored. Error: repetition-operator operand invalid
[info] Found compilation flags in the input bitcode
[info] Gathering functions under test (threads: 1)

       [################################] 1/1.

./build/export/poly_approx/poly_1024_thresh_0_safe-checkpoint.c-amalgamation.c:10:60: error: expected ';' at end of declaration
    float input_array[1] = {0.0f}, output_array[1] = {0.0f}
                                                           ^
                                                           ;
./build/export/poly_approx/poly_1024_thresh_0_safe-checkpoint.c-amalgamation.c:11:64: error: expected ';' at end of declaration
    k2c_tensor input_tensor = {&input_array[0],1,1,{1,1,1,1,1}}
                                                               ^
                                                               ;
./build/export/poly_approx/poly_1024_thresh_0_safe-checkpoint.c-amalgamation.c:12:66: error: expected ';' at end of declaration
    k2c_tensor output_tensor = {&output_array[0],1,1,{1,1,1,1,1}}
                                                                 ^
                                                                 ;
./build/export/poly_approx/poly_1024_thr



clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line bin/intermediate/verifier_stubs.o bin/intermediate/poly_1024_thresh_0_safe-checkpoint.c-amalgamation.o -Iincludes -Inetworks -lm -o bin/poly_approx/poly_1024_thresh_0_safe-checkpoint.c-amalgamation

Compilation Complete: bin/poly_approx/poly_1024_thresh_0_safe-checkpoint.c-amalgamation

Building 99 ./build/export/poly_approx/poly_1024_thresh_0_safe.c-amalgamation.c

clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line -c includes/verifier_stubs.c -Iincludes -Inetworks -o bin/intermediate/verifier_stubs.o
[info] Using configuration /home/yiannis/git/plain_c_nn_benchmark/mull.yml
[warning] Invalid regex for exclude-path: '^(?=.*\/(networks)\/).*$' has been ignored. Error: repetition-operator operand invalid
[info] Found compilation flags in the input bitcode
[info] Gathering functions under test (threads: 1)

       [######

./build/export/poly_approx/poly_16_16_thresh_1_safe-checkpoint.c-amalgamation.c:10:60: error: expected ';' at end of declaration
    float input_array[1] = {0.0f}, output_array[1] = {0.0f}
                                                           ^
                                                           ;
./build/export/poly_approx/poly_16_16_thresh_1_safe-checkpoint.c-amalgamation.c:11:64: error: expected ';' at end of declaration
    k2c_tensor input_tensor = {&input_array[0],1,1,{1,1,1,1,1}}
                                                               ^
                                                               ;
./build/export/poly_approx/poly_16_16_thresh_1_safe-checkpoint.c-amalgamation.c:12:66: error: expected ';' at end of declaration
    k2c_tensor output_tensor = {&output_array[0],1,1,{1,1,1,1,1}}
                                                                 ^
                                                                 ;
./build/export/poly_approx/poly_16_16



clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line bin/intermediate/verifier_stubs.o bin/intermediate/poly_16_16_thresh_1_safe-checkpoint.c-amalgamation.o -Iincludes -Inetworks -lm -o bin/poly_approx/poly_16_16_thresh_1_safe-checkpoint.c-amalgamation

Compilation Complete: bin/poly_approx/poly_16_16_thresh_1_safe-checkpoint.c-amalgamation

Building 160 ./build/export/poly_approx/poly_16_16_thresh_0_safe.c-amalgamation.c

clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line -c includes/verifier_stubs.c -Iincludes -Inetworks -o bin/intermediate/verifier_stubs.o
[info] Using configuration /home/yiannis/git/plain_c_nn_benchmark/mull.yml
[warning] Invalid regex for exclude-path: '^(?=.*\/(networks)\/).*$' has been ignored. Error: repetition-operator operand invalid
[info] Found compilation flags in the input bitcode
[info] Gathering functions under test (threads: 1)

       [#

KeyboardInterrupt: 

#### Delete Intermediate Build Files

In [ ]:
if os.path.exists("./bin/intermediate/"):
    shutil.rmtree("./bin/intermediate/")
    print(f"Cleaned up /bin/intermediate/ intermediate files.")

### Generating Mutation Patches

Now that the mutations have been inserted into the compiled base samples, we have the mutation fuzzed samples. The mutation fuzzed samples can then be **executed** to produce the patches. The _killed_ patches are ones that cause the program execution to defer.

> #### WARNING! 
> Running the following section will execute code on the system. Make sure that the code is safe to run.

In [6]:
def gen_mutation(sample_name: str) -> None:
    print(os.popen(f"mull-runner-15 --reporters Patches --report-dir patches {sample_name}").read())

In [7]:
gen_mutation("bin/poly_approx/poly_16_16_16_16_thresh_5_unsafe.c-amalgamation")

[info] Using config /home/yiannis/git/plain_c_nn_benchmark/mull.yml
[warning] Could not find dynamic library: libm.so.6
[warning] Could not find dynamic library: libc.so.6
[warning] Invalid regex for exclude-path: '^(?=.*\/(networks)\/).*$' has been ignored. Error: repetition-operator operand invalid
[info] Warm up run (threads: 1)

       [################################] 1/1. Finished in 40ms
[info] Filter mutants (threads: 1)

       [################################] 1/1. Finished in 1ms
[info] Baseline run (threads: 1)

       [################################] 1/1. Finished in 45ms
[info] Running mutants (threads: 16)

       [--------------------------------] 2/445
       [--------------------------------] 8/445
       [--------------------------------] 9/445
       [#-------------------------------] 17/445
       [#-------------------------------] 20/445
       [#-------------------------------] 22/445
       [#-------------------------------] 27/445
       [##----------------